In [ ]:
def get_box(file):
    x, y, z = [], [], []
    x, y, z = file['x'],file['y'],file['z']
    x1,y1,z1 = min(x), min(y), min(z)
    x2,y2,z2 = max(x), max(y), max(z)
    return(x1,x2,y1,y2,z1,z2)

def length_square(A1,A2):
    A1,A2 = list(A1),list(A2)
    x1,x2,y1,y2,z1,z2 = float(A1[3]),float(A2[3]),float(A1[4]),float(A2[4]),float(A1[5]),float(A2[5])
    dx = x1 - x2
    dy = y1 - y2
    dz = z1 - z2
    length2 = dx**2+dy**2+dz**2
    return length2

def findTOPinlinker(df,defined_atom_type):
    matching_indices = df[df['Atom_label'] == defined_atom_type].index.tolist()
    count_ATOMS = len(matching_indices)
    print('\n'+str(count_ATOMS)+' top points in ligand file')
    return matching_indices

def findtop_frame(df,defined_ATOM):
    matching_indices = df[df['Atom_label'] == defined_ATOM].index.tolist()
    count_ATOMS = len(matching_indices)
    print('\n'+str(count_ATOMS)+' top points in framework')
    return count_ATOMS

def get_linker_number(list,length):
    count = 0
    for i in list:
        if (length-1) < i <(length+1):
            count += 1
    return count


def find_points_in_radius(points, center, radius):
    points_identity = points.iloc[:, [0,1,2,5]]
    points_positions = np.array(points.loc[:, ['x','y','z']])
    center = np.array(center)
    distances = np.linalg.norm(points_positions - center, axis=1)
    indices = np.where(distances <= radius)[0]
    return points[indices]

def search_unique_vector(df1,MM_l):
    vector = []
    points_n = df1.shape[0]
    for i in range(points_n):            
        for j in range(points_n):
            x = round(length_square(df1.loc[i],df1.loc[j]))
            if ( x == round(MM_l)):
                point_A_frame =  np.asarray(df1.loc[i,['x','y','z']],dtype = float)
                point_B_frame =  np.asarray(df1.loc[j,['x','y','z']],dtype = float)
                v = point_B_frame -point_A_frame
                vAB = np.round(v, 1)
                vector.append(vAB)
    unique_vector = np.unique(vector, axis=0, return_index=False)
    unique_vector = pd.DataFrame(unique_vector,columns=['vx','vy','vz'])
    return unique_vector


def search_neighbor_vector(df1,i,MM_l):
    neighbor_vector = []
    points_n = df1.shape[0]         
    for j in range(points_n):
        x = round(length_square(df1.loc[i],df1.loc[j]))
        if ( x == round(MM_l)):
            point_A_frame =  np.asarray(df1.loc[i,['x','y','z']],dtype = float)
            point_B_frame =  np.asarray(df1.loc[j,['x','y','z']],dtype = float)
            v = point_B_frame -point_A_frame
            vAB = np.round(v, 1)
            neighbor_vector.append(vAB)
    neighbor_vector = pd.DataFrame(neighbor_vector,columns=['vx','vy','vz'])
    return neighbor_vector

def filtered_term_vector(df1,MM_l):
    points_n = df1.shape[0] 
    term_vector = []
    true_indices = []
    for i in range(points_n):
        unique_vector = search_unique_vector(df1,MM_l)
        neighbor_vector = search_neighbor_vector(df1,i,MM_l)
        df_vector = pd.concat([unique_vector,neighbor_vector],ignore_index=False)
        filtered_vector = df_vector.drop_duplicates(keep=False)
        term_vector.append(filtered_vector.to_numpy())
        true_indices.append(filtered_vector.index)
    return [term_vector,true_indices]


def get_box_dimension(file):
    x1,x2,y1,y2,z1,z2 = get_box(file)
    dx,dy,dz = abs(x1-x2), abs(y1-y2), abs(z1-z2)
    dimension = [str(round(dx,5)), str(round(dy,5)),str(round(dz,5))]
    s = ' '.join(dimension)
    return(s)


def calculate_cos(v1,v2):
    cos = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)) if np.linalg.norm(v2) != 0 else None
    return cos

In [ ]:
'''too slow '''
# use a matrix to find body diagnol describe saturation of points 
def points_connectivity_in_box(points,body_diag):
    points_connect_matrix = np.zeros((points.shape[0],points.shape[0]))
    points_along_dx_matrix = np.zeros((points.shape[0],points.shape[0]))
    for i in range(points.shape[0]):
        for j in range(i,points.shape[0]):
            ij = points[j]-points[i]
            if  (np.abs(ij)==body_diag).all(): 
                points_connect_matrix[i][j] = 1
    return points_connect_matrix,points_along_dx_matrix
points_connect_matrix,points_along_dx_matrix= points_connectivity_in_box(points,body_diag)
print(points_connect_matrix)
pairs_body_diag_matrix = np.argwhere(points_connect_matrix)
pairA1_index = pairs_body_diag_matrix[:,0]
pairA2_index = pairs_body_diag_matrix[:,1]
body_centers = np.reshape((points[pairA1_index[0]]+points[pairA2_index[0]])/2,(1,3))
for i in range(1,len(pairA1_index)):
    body_center = (points[pairA1_index[i]]+points[pairA2_index[i]])/2
    body_center = np.reshape(body_center,(1,3))
    body_centers=np.concatenate((body_centers,body_center),axis =0)
unique_body_center = np.unique((body_centers),axis = 0)
#print(body_centers,unique_body_center)


#NOTE: to be removed this cell is for searching by distance criteria
distance_points_array = distance_points_array
unique_distance = np.unique(distance_points_array).tolist()
print(unique_distance)
indices = np.where(distance_points_array == unique_distance[6]) #TODO:variable
pairAB=[]
for i in range(indices[0].shape[0]):
    pairAB.append(np.array([points[indices[0][i]],points[indices[1][i]]]))
print(pairAB)


##TODO:pairAB[i][1][0]
#A_res_Al=df1_node[(df1_node['x']==pairAB[0][0][0])&(df1_node['y']==pairAB[0][0][1])&(df1_node['z']==pairAB[0][0][2])             
#               ].reset_index(drop=True)
#
#A_resnum = A_res_Al.loc[0,'Res_number']
#
#B_res_Al=df1_node[(df1_node['x']==pairAB[0][1][0])&(df1_node['y']==pairAB[0][1][1])&(df1_node['z']==pairAB[0][1][2])             
#               ].reset_index(drop=True)
#B_resnum = B_res_Al.loc[0,'Res_number']
#
#A_res = df1_node[df1_node['Res_number']==A_resnum].reset_index(drop=True)
#B_res = df1_node[df1_node['Res_number']==B_resnum].reset_index(drop=True)
#v_AB = B_res.loc[0, ['x','y','z']].to_numpy()-A_res.loc[0, ['x','y','z']].to_numpy()
##print(A_res,B_res,v_AB)
#AB_res = pd.concat([A_res,B_res],axis =0,join='outer')
#
#def calculate_distance(A,B,X):
#   AX=X-A
#   l_AB=np.linalg.norm(B-A)
#   vAB = (B-A)/l_AB
#   Y=np.dot(AX,vAB)*vAB+A
#   d = np.linalg.norm(Y-X)
#   return d
#
#def add_distance2residue(df,A,B):
#   xyz = np.asarray(df.loc[:,['x','y','z']])
#   d_df= pd.DataFrame(np.zeros((xyz.shape[0], 1)),columns = ['distance'])
#   for i in range(xyz.shape[0]):
#       d_df.loc[i,'distance'] = calculate_distance(A,B,xyz[i])
#   df = pd.concat([df.reset_index(drop = True),d_df],axis = 1, join = 'outer')
#   return df
#
#AB_res = add_distance2residue(AB_res,pairAB[0][0],pairAB[0][1])
#distance_list_of_surroundings = []
#distance_list_of_surroundings = AB_res['distance'].to_list()
#distance_list_of_surroundings.sort()
#print(distance_list_of_surroundings[2:6])
#AB_res[(AB_res['distance']>0.9*distance_list_of_surroundings[2])&(AB_res['distance']<1.1*distance_list_of_surroundings[5])]
#guide_atoms_in_AB = AB_res[(AB_res['distance']>0.9*distance_list_of_surroundings[2])
#                           &(AB_res['distance']<1.1*distance_list_of_surroundings[5])].reset_index(drop=True)
#print(guide_atoms_in_AB)


In [ ]:
'''
quaternion
'''

def calculate_q_rotation_with_axis_degree(axis,theta): #axis is HE---HE ,theta from O1--AXIS--O1'
    w = theta/2
    s = np.sin(w)
    q_real= np.array([np.cos(w)])
    q_ijk = s*axis
    q_r = np.concatenate([q_real,q_ijk])
    q_r = quaternion.from_float_array(q_r)
    return q_r


def calculate_angle_rad(axis,p1, p2):
    axis = normalize_vector(axis)
    a_square=np.linalg.norm(p1)*np.linalg.norm(p1)-np.dot(p1,axis)*np.dot(p1,axis)
    b_square=np.linalg.norm(p2)*np.linalg.norm(p2)-np.dot(p2,axis)*np.dot(p2,axis)
    c_square=np.linalg.norm(p2-p1)*np.linalg.norm(p2-p1)
    a,b = np.sqrt(a_square),np.sqrt(b_square)
    cos_theta = (a_square+b_square-c_square)/(2*a*b)
    print(cos_theta,a_square,b_square,c_square,axis,p1,p2)
    cos_theta = np.clip(cos_theta,-1,1)
    theta_rad = np.arccos(cos_theta)
    print(theta_rad)
    return theta_rad

In [ ]:
'''AB points searching'''

def groupA_one_step_to_groupB(first_B,group_A,d,points):
    point_dx = group_A+d
    group_B = first_B
    for i in range(group_A.shape[0]):
        point = np.reshape(point_dx[i], (1, 3))
        group_B = np.concatenate((group_B,point),axis = 0) if np.all(points == point, axis=1).any() else group_B
        group_B = np.unique(group_B,axis=0)
    return group_B

def groupA_one_step_to_groupA(group_A,d,points):
    point_dx = group_A+d
    for i in range(group_A.shape[0]):
        point = np.reshape(point_dx[i], (1, 3))
        group_A = np.concatenate((group_A,point),axis = 0) if np.all(points == point, axis=1).any() else group_A
        group_A = np.unique(group_A,axis=0)
    return group_A

def group_points_AB(x_num,y_num,z_num,dx_value,dy_value,dz_value):
   points=points_generator(x_num,y_num,z_num,dx_value,dy_value,dz_value)
   O = np.array([[0,0,0]])
   dx = dx_value*np.array([[1,0,0]]) #dx_value works as a scalar
   dy = dy_value*np.array([[0,1,0]])
   dz = dz_value*np.array([[0,0,1]])

   first_A = O
   first_B = O+dy
   group_A = first_A
   group_B = first_B

   for i in range(points.shape[0]):
      if group_A.shape[0]+group_B.shape[0] == points.shape[0]:
         break
      else:
         group_B_dx = groupA_one_step_to_groupB(first_B,group_A,dx,points)
         group_B_dy = groupA_one_step_to_groupB(first_B,group_A,dy,points)
         group_B_dz = groupA_one_step_to_groupA(group_B,dz,points)
         group_B_dxyz = [group_B,group_B_dx,group_B_dy,group_B_dz]
         group_B = np.concatenate([arr for arr in group_B_dxyz if arr is not None],axis = 0) 
         group_B = np.unique(group_B,axis=0)
         group_A_dx = groupA_one_step_to_groupB(first_A,group_B,dx,points)
         group_A_dy = groupA_one_step_to_groupB(first_A,group_B,dy,points)
         group_A_dz = groupA_one_step_to_groupA(group_A,dz,points)
         group_A_dxyz = [group_A,group_A_dx,group_A_dy,group_A_dz]
         group_A = np.concatenate([arr for arr in group_A_dxyz if arr is not None],axis = 0)
         group_A = np.unique(group_A,axis=0)
   
   return group_A,group_B


def get_center_point_of_face(p1_face,p2_face,p3_face):
    center_point = (normalize_vector(p1_face)+
                              normalize_vector(p2_face)+
                              normalize_vector(p2_face))/3
    return center_point